# Preventing Prompt Injection Attacks with Amazon Bedrock AgentCore Gateway Interceptors

## Overview

This notebook demonstrates how to **prevent prompt injection attacks** using **Amazon Bedrock AgentCore Gateway interceptors** with **LLM-based prompt analysis**. The interceptor examines incoming user prompts before they reach tools, using a large language model to detect potential injection attempts and block malicious inputs before they can cause harm.

In this tutorial, we focus specifically on **SQL injection prevention** as a practical example, but the same pattern can be applied to detect and prevent other types of prompt injection attacks including:
- Command injection
- Jailbreak attempts
- Prompt leaking
- Role manipulation
- Context poisoning

### Why Prevent Prompt Injection at the Gateway?

When building AI agents that interact with sensitive systems, prompt injection remains a critical security threat:

- **Proactive Defense**: Block malicious prompts before they reach your tools and systems
- **Centralized Security**: Apply injection detection consistently across all tools
- **AI-Powered Detection**: Use LLMs to identify sophisticated injection patterns that traditional regex can't catch
- **Zero Trust Architecture**: Don't rely on downstream tools to sanitize inputs
- **Extensible Pattern**: Adapt the same approach for different injection types
- **Compliance**: Meet security requirements for system access controls

The Gateway interceptor provides a **centralized enforcement point** that validates prompts before any tool executes, without modifying individual tool implementations.

---

## What This Tutorial Covers

This tutorial implements prompt injection prevention using a **REQUEST interceptor** with **LLM-based analysis**:

🛡️ **Prompt Injection Prevention (REQUEST interceptor + LLM Analysis)**  
   - Intercepts incoming requests before they reach tools
   - Analyzes user prompts using an LLM to detect injection patterns
   - **Demo Focus**: SQL injection detection (keywords, union attacks, comment injections, obfuscation)
   - Blocks suspicious requests and returns security warnings
   - Allows legitimate queries to proceed to tools
   - **Extensible**: Same pattern works for other injection types

![Prompt Injection Prevention Architecture](images/prompt-injection-prevention.png)

> **Note:** Architecture diagram shows: User → Gateway → REQUEST Interceptor (LLM Analysis) → Tool (Mock DB). Interceptor blocks malicious requests before they reach tools.

---

## Why Use Gateway Interceptors?

Gateway Interceptors allow you to:

- **Prompt Validation**: Analyze user inputs before they reach sensitive tools
- **Intelligent Detection**: Use LLMs to identify sophisticated attack patterns
- **Flexible Security**: Adapt detection logic without changing tool implementations
- **Audit & Monitoring**: Log all security events and blocked attempts
- **Request Transformation**: Sanitize or reject requests in transit
- **Multi-Threat Protection**: Apply different detection rules for different threat types
- **Extensible Pattern**: Easily adapt to new injection attack vectors

Because interceptors are attached at the **Gateway layer**, they protect **any** underlying tool or MCP server without modifying application code.

---

## Tutorial Details

| Information              | Details                                                                      |
|--------------------------|------------------------------------------------------------------------------|
| **Tutorial type**        | Interactive                                                                  |
| **AgentCore components** | Amazon Bedrock AgentCore Gateway, Gateway Interceptors                      |
| **Gateway Target type**  | MCP Server (Lambda-based database tool)                                     |
| **Interceptor types**    | AWS Lambda (REQUEST)                                                        |
| **Inbound Auth IdP**     | Amazon Cognito (CUSTOM_JWT authorizer)                                      |
| **Security Pattern**     | Prompt injection detection using LLM-based analysis (SQL injection demo)    |
| **Tutorial components**  | Gateway, Lambda Interceptor, Amazon Bedrock, Amazon Cognito, MCP tools      |
| **Tutorial vertical**    | Cross-vertical (applicable to any AI agent with tool access)                |
| **Example complexity**   | Intermediate                                                                 |
| **SDK used**             | boto3                                                                        |

---

## Prerequisites

To execute this tutorial you will need:

- Jupyter notebook (Python kernel)
- AWS credentials with permissions for:
  - AWS Lambda
  - AWS IAM
  - Amazon Cognito
  - Amazon Bedrock AgentCore services (control plane)
  - Amazon Bedrock (bedrock:InvokeModel for LLM analysis)
- Python 3.9 or higher
- Basic understanding of AWS Lambda, IAM roles, Amazon Cognito, Amazon Bedrock, and Amazon Bedrock AgentCore Gateway

> ⚠️ **Note:** The Cleanup section at the end deletes the AWS resources created by this tutorial (Gateway, Lambdas, IAM roles, etc.). Only run it when you're ready to tear everything down.


---

## Part 1: Setup & Deployment

### Step 1.0: Install Required Dependencies

Install all necessary Python packages for this tutorial.

In [ ]:
!pip install -r requirements.txt

### Step 1.1: Import Required Libraries

In [ ]:
import boto3
import json
import time
import sys
from pathlib import Path
from datetime import datetime
from botocore.exceptions import ClientError

# Add parent directory to path for utils
utils_dir = Path.cwd().parent
sys.path.insert(0, str(utils_dir))

import utils

print("✓ Libraries imported")

# Generate unique identifier for this deployment
DEPLOYMENT_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"\nDeployment ID: {DEPLOYMENT_ID}")

### Step 1.2: Configure Deployment Variables

In [ ]:
# Configuration
REGION = "us-east-1"  
LAMBDA_FUNCTION_NAME = f"interceptor-lambda-{DEPLOYMENT_ID}"
LAMBDA_ROLE_NAME = f"interceptor-lambda-role-{DEPLOYMENT_ID}"
GATEWAY_NAME = f"interceptor-gateway-{DEPLOYMENT_ID}"

# Initialize clients
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
cognito_client = boto3.client('cognito-idp', region_name=REGION)

print("Configuration:")
print(f"  Lambda Function: {LAMBDA_FUNCTION_NAME}")
print(f"  Lambda Role: {LAMBDA_ROLE_NAME}")
print(f"  Gateway Name: {GATEWAY_NAME}")
print(f"  Region: {REGION}")


### Step 1.3: Configure LLM Model for Prompt Injection Detection

Configure the Amazon Bedrock model that will analyze prompts for injection patterns. We'll use Claude Haiku for fast, cost-effective security analysis.

**Model Selection:**
- **Claude 3.5 Haiku**: Fast inference, excellent at security analysis, cost-effective
- **Alternative**: Amazon Nova Micro for even faster/cheaper analysis

**Pricing:** See [Amazon Bedrock Pricing](https://aws.amazon.com/bedrock/pricing/) for current model costs.

In this demo, we focus on SQL injection detection, but the same model can be adapted for other injection types.

In [ ]:
# Configure LLM model for prompt injection detection
print("Configuring LLM model for prompt injection detection...")

# Model configuration - using Claude Haiku for fast, cost-effective analysis
MODEL_ID = "anthropic.claude-3-5-haiku-20241022:0"  # Fast and efficient for security analysis
# Alternative: "amazon.nova-micro-v1:0" for even faster responses

print(f"✓ Using model: {MODEL_ID}")
print(f"  Purpose: Analyze user prompts for injection attack patterns")
print(f"  Pricing: See https://aws.amazon.com/bedrock/pricing/ for current costs")
print(f"\n  Demo Focus: SQL Injection Detection")
print(f"    - SQL keywords (SELECT, DROP, UNION, etc.)")
print(f"    - Comment-based injections (-- , /* */)")
print(f"    - Boolean-based attacks (OR 1=1, AND 1=1)")
print(f"    - String concatenation attacks")
print(f"    - Obfuscation techniques")
print(f"    - Time-based blind SQL injection")
print(f"    - Stacked queries")
print(f"\n  Extensible to Other Injection Types:")
print(f"    - Command injection (shell commands, system calls)")
print(f"    - Jailbreak attempts (role manipulation, instruction override)")
print(f"    - Prompt leaking (system prompt extraction)")
print(f"    - Context poisoning (malicious context injection)")
print(f"\n  Security: Fails CLOSED - blocks requests if LLM analysis fails")

# Store model configuration for Lambda
PROMPT_INJECTION_MODEL_ID = MODEL_ID


### Step 1.4: Create IAM Role for Lambda Interceptor

Grant Lambda permissions to execute, write CloudWatch logs, and invoke Bedrock models for prompt injection analysis.

In [ ]:
# Create IAM role for Lambda interceptor using utils
print("Creating IAM role for Lambda interceptor...")

LAMBDA_ROLE_ARN = utils.create_lambda_role(
    role_name=LAMBDA_ROLE_NAME,
    description='Role for AgentCore Lambda Interceptor for prompt injection prevention with Bedrock'
)

print(f"  ARN: {LAMBDA_ROLE_ARN}")

# Add Bedrock InvokeModel permissions to Lambda role
print("\nAdding Bedrock InvokeModel permissions to Lambda role...")
iam_client = boto3.client('iam')

bedrock_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": "*"
        }
    ]
}

try:
    iam_client.put_role_policy(
        RoleName=LAMBDA_ROLE_NAME,
        PolicyName='BedrockInvokeModelPolicy',
        PolicyDocument=json.dumps(bedrock_policy)
    )
    print("✓ Bedrock InvokeModel permissions added")
    print("  Policy: bedrock:InvokeModel on all resources")
except Exception as e:
    print(f"⚠ Failed to add Bedrock permissions: {e}")

### Step 1.4a: Wait for IAM Role to Propagate

Allow time for the IAM role and policies to propagate before deploying the Lambda function.

In [ ]:
time.sleep(10)

### Step 1.5: Deploy Lambda Interceptor Function

Lambda intercepts incoming requests and analyzes prompts for injection patterns using an LLM before allowing them to reach tools. In this demo, we focus on SQL injection detection.

In [ ]:
# Deploy Lambda interceptor using utils
print("Deploying Lambda interceptor...")

# Prepare environment variables for Lambda
lambda_env_vars = {}
if 'PROMPT_INJECTION_MODEL_ID' in globals():
    lambda_env_vars['MODEL_ID'] = PROMPT_INJECTION_MODEL_ID
    print(f"  Configuring Lambda with Model: {PROMPT_INJECTION_MODEL_ID}")
else:
    print("  ⚠ WARNING: MODEL_ID not found. Lambda will use default model.")
    print("  Make sure to run Step 1.3 to configure the model first.")

LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=LAMBDA_FUNCTION_NAME,
    role_arn=LAMBDA_ROLE_ARN,
    lambda_code_path='src/lambda/lambda_function.py',
    description='AgentCore Request Lambda Interceptor to prevent prompt injection using LLM analysis',
    timeout=60,  # Increased for LLM analysis time
    memory_size=256,
    environment_vars=lambda_env_vars if lambda_env_vars else None,
    region=REGION
)

print(f"  ARN: {LAMBDA_ARN}")

### Step 1.5a: Grant Gateway Permission to Invoke Lambda

Add permissions for the Gateway to invoke the Lambda interceptor function.

In [ ]:
# Grant Gateway permission to invoke the Lambda interceptor
print("\nGranting Gateway permission to invoke Lambda...")

utils.grant_gateway_invoke_permission(
    function_name=LAMBDA_FUNCTION_NAME,
    region=REGION
)

### Step 1.6: Create Amazon Cognito User Pool & App Client

Create Cognito user pool for Gateway authentication using OAuth client credentials flow.

In [ ]:
# Create Cognito User Pool and Client for Gateway authentication using utils
print("Creating Cognito User Pool and Client...")

USER_POOL_NAME = f"gateway-pool-{DEPLOYMENT_ID}"
RESOURCE_SERVER_ID = 'gateway'
RESOURCE_SERVER_NAME = 'Gateway Resource Server'
SCOPES = [{'ScopeName': 'tools', 'ScopeDescription': 'Access to gateway tools'}]

# Create or get user pool
USER_POOL_ID = utils.get_or_create_user_pool(cognito_client, USER_POOL_NAME)
print(f"  Pool ID: {USER_POOL_ID}")

# Create or get resource server
utils.get_or_create_resource_server(cognito_client, USER_POOL_ID, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)

# Wait for resource server to propagate
print("  Waiting for resource server to propagate...")
time.sleep(3)

# Create M2M client with client credentials flow
CLIENT_NAME = f"gateway-client-{DEPLOYMENT_ID}"
CLIENT_ID, CLIENT_SECRET = utils.get_or_create_m2m_client(
    cognito_client,
    USER_POOL_ID,
    CLIENT_NAME,
    RESOURCE_SERVER_ID,
    SCOPES=[f"{RESOURCE_SERVER_ID}/tools"]
)

print(f"✓ User Pool Client created: {CLIENT_NAME}")
print(f"  Client ID: {CLIENT_ID}")
print(f"  Client Secret: {CLIENT_SECRET[:20]}...")

# Construct OAuth URLs
POOL_DOMAIN = USER_POOL_ID.replace('_', '').lower()
COGNITO_DOMAIN = f"https://{POOL_DOMAIN}.auth.{REGION}.amazoncognito.com"
DISCOVERY_URL = f"https://cognito-idp.{REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"
TOKEN_URL = f"{COGNITO_DOMAIN}/oauth2/token"

print(f"\n✓ OAuth Configuration:")
print(f"  Discovery URL: {DISCOVERY_URL}")
print(f"  Token URL: {TOKEN_URL}")
print(f"  Scope: {RESOURCE_SERVER_ID}/tools")

### Step 1.7: Create Gateway with Request Interceptor

**Why REQUEST Interceptor?**  
The interceptor processes incoming requests before they reach tools, allowing us to analyze and block prompt injection attempts (including SQL injection) before any tool executes.

In [ ]:
# Create Gateway IAM role
gateway_iam_role = utils.create_agentcore_gateway_role_with_region(GATEWAY_NAME, REGION)
GATEWAY_ROLE_ARN = gateway_iam_role['Role']['Arn']

print(f"✓ Gateway role created: {GATEWAY_ROLE_ARN}")

# Wait for role propagation
time.sleep(10)

# Create Gateway with Lambda interceptor
print(f"\nCreating Gateway with REQUEST interceptor...")

try:
    gateway_response = gateway_client.create_gateway(
        name=GATEWAY_NAME,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26"]
            }
        },
        interceptorConfigurations=[
            {
                "interceptor": {
                    "lambda": {
                        "arn": LAMBDA_ARN
                    }
                },
                "interceptionPoints": ["REQUEST"],
                "inputConfiguration": {
                    "passRequestHeaders": True  
                }
            }
        ],
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": DISCOVERY_URL,
                "allowedClients": [CLIENT_ID]
            }
        },
        roleArn=GATEWAY_ROLE_ARN
    )
    
    GATEWAY_ID = gateway_response.get('gatewayId')
    print(f"✓ Gateway created: {GATEWAY_ID}")
    
except Exception as e:
    print(f"\n✗ Failed to create Gateway: {e}")
    raise


### Step 1.8: Wait for Gateway to be Ready

In [ ]:
# Wait for Gateway to be ready using signed requests
print("\nWaiting for Gateway to be ready...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
        status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

        if status_code == 200:
            # gateway_info = response.json()
            status = response.get('status', 'UNKNOWN')
            
            print(f"  [{attempt + 1}/{max_attempts}] Status: {status}")
            
            if status == 'READY':
                GATEWAY_URL = response.get('gatewayUrl')
                print(f"\n✓ Gateway is ready!")
                print(f"  URL: {GATEWAY_URL}")
                
                # Show interceptor configuration
                if 'interceptorConfigurations' in response:
                    interceptor_configs = response['interceptorConfigurations']
                    print(f"\n  Interceptor Configuration:")
                    for idx, config in enumerate(interceptor_configs):
                        print(f"    [{idx}] Interception Points: {config.get('interceptionPoints', [])}")
                        print(f"    [{idx}] Lambda ARN: {config.get('interceptor', {}).get('lambda', {}).get('arn', 'N/A')}")
                        print(f"    [{idx}] Pass Headers: {config.get('inputConfiguration', {}).get('passRequestHeaders', False)}")
                break
            elif status == 'FAILED':
                print(f"\n✗ Gateway creation failed")
                print(f"  Details: {response}")
                raise Exception("Gateway failed")
        else:
            print(f"  [{attempt + 1}/{max_attempts}] HTTP Error: {response.status_code}")
    except Exception as e:
        print(f"  [{attempt + 1}/{max_attempts}] Error: {e}")
    
    time.sleep(10)
else:
    print(f"\n⚠ Timeout waiting for Gateway")
    raise Exception("Gateway timeout")


### Step 1.9: Register Sample Database Tools with Gateway

Deploy sample database tool Lambda (customer query tool) and register it as a Gateway target.

**Note:** This tool uses mock data - no real database required. It simulates what would happen with a real database query interface.

In [ ]:
# Deploy tool Lambdas and register as Gateway targets
print("Deploying tool Lambda functions...")

# Import tool modules
sys.path.insert(0, str(Path.cwd()))
from src.tools import customer_query_tool

# Create IAM role for tool Lambdas using utils
TOOL_ROLE_ARN = utils.create_lambda_role(
    role_name=f"tool-lambda-role-{DEPLOYMENT_ID}",
    description='Role for tool Lambda functions'
)

# Deploy tool Lambda functions
tools_to_deploy = [
    ('customer_query_tool', customer_query_tool),
]

deployed_tools = []

for tool_name, tool_module in tools_to_deploy:
    print(f"  Deploying {tool_name}...")
    
    function_name = f"{tool_name.replace('_', '-')}-{DEPLOYMENT_ID}"
    tool_code_path = Path(tool_module.__file__)
    
    lambda_arn = utils.deploy_lambda_function(
        function_name=function_name,
        role_arn=TOOL_ROLE_ARN,
        lambda_code_path=str(tool_code_path),
        environment_vars={'TOOL_NAME': tool_name},
        description=f'{tool_name} function - mock database query tool',
        region=REGION
    )
    
    tool_definition = getattr(tool_module, 'TOOL_DEFINITION', {
        "name": tool_name,
        "description": f"{tool_name} function"
    })
    
    deployed_tools.append({
        'tool_name': tool_name,
        'function_name': function_name,
        'lambda_arn': lambda_arn,
        'tool_definition': tool_definition
    })

print(f"✓ Deployed {len(deployed_tools)} tool Lambdas")

# Register tools as Gateway targets
print("\nRegistering tools as Gateway targets...")
created_targets = []

for tool in deployed_tools:
    print(f"  Registering {tool['tool_name']}...")
    
    try:
        response = gateway_client.create_gateway_target(
            gatewayIdentifier=GATEWAY_ID,
            name=f"{tool['tool_name'].replace('_', '-')}-target",
            targetConfiguration={
                "mcp": {
                    "lambda": {
                        "lambdaArn": tool["lambda_arn"],
                        "toolSchema": {"inlinePayload": [tool["tool_definition"]]}
                    }
                }
            },
            credentialProviderConfigurations=[{
                "credentialProviderType": "GATEWAY_IAM_ROLE"
            }]
        )
        
        target_id = response['targetId']
        print(f"    ✓ Target created: {target_id}")
        
        # Wait for target to be READY
        for attempt in range(18):
            status_response = gateway_client.get_gateway_target(
                gatewayIdentifier=GATEWAY_ID,
                targetId=target_id
            )
            status = status_response.get('status')
            
            if status == 'READY':
                print(f"    ✓ Target is READY")
                created_targets.append({
                    'tool_name': tool['tool_name'],
                    'target_id': target_id,
                    'lambda_arn': tool['lambda_arn']
                })
                break
            elif status == 'FAILED':
                print(f"    ✗ Target FAILED")
                break
            
            time.sleep(10)
            
    except Exception as e:
        print(f"    ✗ Failed to create target: {e}")

# Summary
print(f"\n✓ Deployed {len(deployed_tools)} tool Lambdas")
print(f"✓ Created {len(created_targets)} gateway targets")

if len(created_targets) < len(deployed_tools):
    print(f"⚠ Warning: Not all targets were created successfully")

# Store for cleanup
DEPLOYED_TOOL_FUNCTIONS = [t['function_name'] for t in deployed_tools]
CREATED_TARGET_IDS = [t['target_id'] for t in created_targets]


---

## Part 2: Testing

### Step 2.1: Test Prompt Injection Prevention with LLM Analysis

Test the interceptor with both legitimate queries and injection attempts to verify that malicious prompts are blocked. In this demo, we focus on SQL injection as a specific example of prompt injection.

#### What to Expect:

The Lambda interceptor will:

1. **Intercept the request** before it reaches the tool
2. **Extract the user prompt** from the request parameters
3. **Analyze the prompt using an LLM** to detect injection patterns
4. **Block malicious requests** and return a security warning
5. **Allow legitimate requests** to proceed to the tool

#### SQL Injection Detection Patterns (Demo Focus):

The LLM analyzes prompts for common SQL injection techniques:

- **SQL Keywords**: `SELECT`, `DROP`, `DELETE`, `UPDATE`, `INSERT`, `UNION`, `ALTER`
- **Comment Injections**: `--`, `/* */`, `#`
- **Boolean Attacks**: `OR 1=1`, `AND 1=1`, `' OR '1'='1`
- **String Concatenation**: `'; DROP TABLE--`, `' UNION SELECT--`
- **Obfuscation**: `CHAR()`, `CONCAT()`, hex encoding
- **Time-Based Blind**: `SLEEP()`, `WAITFOR DELAY`
- **Stacked Queries**: Multiple statements separated by `;`

#### Other Injection Types (Extensible Pattern):

The same interceptor pattern can detect:
- **Command Injection**: Shell commands, system calls
- **Jailbreak Attempts**: Role manipulation, instruction override
- **Prompt Leaking**: System prompt extraction attempts
- **Context Poisoning**: Malicious context injection

#### Example Scenarios:

**Legitimate Query (ALLOWED):**
```
User Prompt: "Show me customer information for customer ID 12345"
Result: ✓ Request proceeds to database tool
```

**SQL Injection Attempt (BLOCKED):**
```
User Prompt: "Show me customer ID 12345' OR '1'='1"
Result: ✗ Request blocked with security warning
Error: "Potential prompt injection detected. Request blocked for security."
```

**Advanced Injection (BLOCKED):**
```
User Prompt: "Get customer 123; DROP TABLE customers--"
Result: ✗ Request blocked with security warning
```



In [ ]:
# Test the prompt injection prevention by invoking the tool
import requests

print("Testing prompt injection prevention interceptor...")
print(f"Demo Focus: SQL Injection Detection")
print(f"Gateway URL: {GATEWAY_URL}")

# Get OAuth token
token_data = utils.get_token(
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope_string="gateway/tools",
    REGION=REGION
)

if 'error' in token_data:
    print(f"✗ Token request failed: {token_data['error']}")
else:
    token = token_data['access_token']
    print(f"✓ Token obtained")

### Step 2.2: Test Legitimate Query (Should Pass)

Test a legitimate customer query that should pass through the interceptor without being blocked.

**Expected Result:**
- The LLM analyzes the prompt and determines it's safe
- The request proceeds to the database tool
- Customer data is returned successfully

In [ ]:
# Test a legitimate query (should pass)
print("\n" + "="*60)
print("Test 1: Legitimate Query (Should PASS)")
print("="*60)

# Reuse the token from previous step
if 'token' in locals():
    print(f"✓ Using existing token")
    
    # Call the database tool with a legitimate query
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 1,
        "params": {
            "name": "customer-query-tool-target___customer_query_tool",
            "arguments": {
                "query": "Show me customer information for customer ID 12345"
            }
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"\n✓ Legitimate query PASSED through interceptor")
        print(f"\nResponse:")
        print(json.dumps(result, indent=2))
        
        print(f"\n📝 Analysis:")
        print(f"  - Query: 'Show me customer information for customer ID 12345'")
        print(f"  - LLM determined this is a legitimate query")
        print(f"  - No SQL injection patterns detected")
        print(f"  - Request proceeded to database tool")
        print(f"  - Tool returned mock customer data")
    else:
        print(f"✗ Request failed: {response.status_code}")
        print(f"Response: {response.text}")
else:
    print("✗ No token available. Please run Step 2.1 first.")

### Step 2.3: Test SQL Injection Attempt #1 - Boolean-Based Attack (Should Block)

Test a classic SQL injection attempt using boolean logic to bypass authentication.

**Expected Result:**
- The LLM detects the SQL injection pattern
- The request is blocked before reaching the tool
- An error response is returned explaining the security threat

In [ ]:
# Test SQL injection attempt - Boolean-based
print("\n" + "="*60)
print("Test 2: SQL Injection Attempt - Boolean-Based (Should BLOCK)")
print("="*60)

if 'token' in locals():
    print(f"✓ Using existing token")
    
    # Attempt SQL injection with OR 1=1
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 2,
        "params": {
            "name": "customer-query-tool-target___customer_query_tool",
            "arguments": {
                "query": "customer ID 12345' OR '1'='1"
            }
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    result = response.json()
    print(f"\nResponse:")
    print(json.dumps(result, indent=2))
    
    # Check if request was blocked
    if 'error' in result:
        print(f"\n✓ SQL injection attempt BLOCKED by interceptor")
        print(f"\n📝 Analysis:")
        print(f"  - Query: \"customer ID 12345' OR '1'='1\"")
        print(f"  - Attack Type: Boolean-based SQL injection")
        print(f"  - LLM detected: OR clause with always-true condition")
        print(f"  - Request blocked before reaching database tool")
        print(f"  - Error returned to user with security explanation")
    else:
        print(f"\n✗ WARNING: SQL injection was NOT blocked!")
        print(f"  This should have been detected as malicious")
else:
    print("✗ No token available. Please run Step 2.1 first.")

### Step 2.4: Test SQL Injection Attempt #2 - Stacked Queries (Should Block)

Test a destructive SQL injection attempt using stacked queries to drop a table.

**Expected Result:**
- The LLM detects the dangerous DROP TABLE command
- The request is blocked immediately
- The database tool never receives the malicious query

In [ ]:
# Test SQL injection attempt - Stacked queries with DROP TABLE
print("\n" + "="*60)
print("Test 3: SQL Injection Attempt - Stacked Queries (Should BLOCK)")
print("="*60)

if 'token' in locals():
    print(f"✓ Using existing token")
    
    # Attempt SQL injection with DROP TABLE
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 3,
        "params": {
            "name": "customer-query-tool-target___customer_query_tool",
            "arguments": {
                "query": "customer 123; DROP TABLE customers--"
            }
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    result = response.json()
    print(f"\nResponse:")
    print(json.dumps(result, indent=2))
    
    # Check if request was blocked
    if 'error' in result:
        print(f"\n✓ SQL injection attempt BLOCKED by interceptor")
        print(f"\n📝 Analysis:")
        print(f"  - Query: \"customer 123; DROP TABLE customers--\"")
        print(f"  - Attack Type: Stacked queries with destructive command")
        print(f"  - LLM detected: Semicolon separator, DROP TABLE, comment syntax")
        print(f"  - Request blocked - database protected from deletion")
        print(f"  - This demonstrates protection against destructive attacks")
    else:
        print(f"\n✗ WARNING: SQL injection was NOT blocked!")
        print(f"  This should have been detected as highly malicious")
else:
    print("✗ No token available. Please run Step 2.1 first.")

### Step 2.5: Test SQL Injection Attempt #3 - UNION-Based Attack (Should Block)

Test a UNION-based SQL injection attempt to extract data from other tables.

**Expected Result:**
- The LLM detects the UNION SELECT pattern
- The request is blocked to prevent data exfiltration
- Security policy is enforced consistently

In [ ]:
# Test SQL injection attempt - UNION-based
print("\n" + "="*60)
print("Test 4: SQL Injection Attempt - UNION-Based (Should BLOCK)")
print("="*60)

if 'token' in locals():
    print(f"✓ Using existing token")
    
    # Attempt SQL injection with UNION SELECT
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 4,
        "params": {
            "name": "customer-query-tool-target___customer_query_tool",
            "arguments": {
                "query": "customer 123' UNION SELECT * FROM users--"
            }
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    result = response.json()
    print(f"\nResponse:")
    print(json.dumps(result, indent=2))
    
    # Check if request was blocked
    if 'error' in result:
        print(f"\n✓ SQL injection attempt BLOCKED by interceptor")
        print(f"\n📝 Analysis:")
        print(f"  - Query: \"customer 123' UNION SELECT * FROM users--\"")
        print(f"  - Attack Type: UNION-based data exfiltration")
        print(f"  - LLM detected: UNION keyword, SELECT statement, comment syntax")
        print(f"  - Request blocked - prevented unauthorized data access")
        print(f"  - This demonstrates protection against data theft attempts")
    else:
        print(f"\n✗ WARNING: SQL injection was NOT blocked!")
        print(f"  This should have been detected as a data exfiltration attempt")
else:
    print("✗ No token available. Please run Step 2.1 first.")

### Step 2.6: Observability and Monitoring

The interceptor logs all security decisions to CloudWatch Logs for monitoring and auditing.

**Best Practices for Production:**

1. **CloudWatch Logs**: All interceptor decisions are logged with `[SECURITY]` and `[INTERCEPTOR]` prefixes
   - View logs in CloudWatch Console under `/aws/lambda/{function-name}`
   - Set up log insights queries to track blocked requests
   - See [CloudWatch Logs Documentation](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html)

2. **SNS Notifications**: Send alerts when injection attempts are detected
   - Add SNS publish call in Lambda when `is_safe = False`
   - Configure SNS topic for security team notifications
   - See [SNS Documentation](https://docs.aws.amazon.com/sns/latest/dg/welcome.html)

3. **Metrics and Alarms**: Track security events over time
   - Create CloudWatch metrics for blocked vs allowed requests
   - Set up alarms for unusual patterns (spike in blocked requests)
   - See [CloudWatch Alarms Documentation](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AlarmThatSendsEmail.html)

4. **Security Hub Integration**: Centralize security findings
   - Send blocked injection attempts to AWS Security Hub
   - Integrate with SIEM systems for correlation
   - See [Security Hub Documentation](https://docs.aws.amazon.com/securityhub/latest/userguide/what-is-securityhub.html)

5. **False Positive Handling**: Tune the LLM prompt to reduce false positives
   - Monitor CloudWatch logs for legitimate queries being blocked
   - Adjust the analysis prompt in Lambda environment variables
   - Implement feedback loop for continuous improvement

---

# Part 3: Cleanup - Delete All Resources

⚠️ **WARNING: This will DELETE all resources created in Part 1!**

Only run this section if you want to clean up everything.

### Step 3.1: Delete Created Resources

In [ ]:
# Cleanup - Delete all created resources using utils
print("Starting cleanup...")

# 1. Delete gateway targets
if 'CREATED_TARGET_IDS' in globals() and 'GATEWAY_ID' in globals():
    utils.delete_gateway_targets(gateway_client, GATEWAY_ID, CREATED_TARGET_IDS)
    # Wait for target deletions to complete before deleting gateway
    time.sleep(5)

# 2. Delete gateway
if 'GATEWAY_ID' in globals():
    utils.delete_gateway(gateway_client, GATEWAY_ID)
    print("✓ Deleted gateway")

# 3. Delete Lambda functions (tools + interceptor)
lambda_functions_to_delete = []
if 'DEPLOYED_TOOL_FUNCTIONS' in globals():
    lambda_functions_to_delete.extend(DEPLOYED_TOOL_FUNCTIONS)
if 'LAMBDA_FUNCTION_NAME' in globals():
    lambda_functions_to_delete.append(LAMBDA_FUNCTION_NAME)

if lambda_functions_to_delete:
    utils.delete_lambda_functions(lambda_functions_to_delete, REGION)

# 4. Delete IAM roles
if 'LAMBDA_ROLE_NAME' in globals():
    utils.delete_iam_role(LAMBDA_ROLE_NAME)
if 'DEPLOYMENT_ID' in globals():
    utils.delete_iam_role(f"tool-lambda-role-{DEPLOYMENT_ID}")
    utils.delete_iam_role(f"agentcore-{GATEWAY_NAME}-role")

# 5. Delete Cognito user pool
if 'USER_POOL_ID' in globals():
    utils.delete_cognito_user_pool(USER_POOL_ID, REGION)

print("\n✓ Cleanup complete!")

---

# Summary

This notebook demonstrates prompt injection prevention using Lambda interceptors:

1. ✅ **Setup** - Created Lambda interceptor, IAM roles, Cognito, and Gateway with REQUEST interception
2. ✅ **Test** - Verified prompt injection detection blocks malicious inputs (SQL injection demo)
3. ✅ **Cleanup** - Deleted all resources

## What We Demonstrated

- **Lambda REQUEST interceptor** that analyzes prompts before they reach tools
- **LLM-based injection detection** using Amazon Bedrock models
- **SQL injection prevention** as a practical demo (extensible to other injection types)
- **Gateway integration** with custom security interceptors
- **Complete resource lifecycle** management

## Next Steps

- Customize the LLM prompt for different injection types (command injection, jailbreaks)
- Add multi-layer detection with different threat models
- Integrate with security information and event management (SIEM) systems
- Monitor CloudWatch logs for security events and blocked attempts